<a href="https://colab.research.google.com/github/isabellasims/Google-colabs/blob/master/geopandas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
!pip install geopandas
import geopandas as gpd #rename as gpd
import math

     |████████████████████████████████| 972kB 2.7MB/s 
     |████████████████████████████████| 10.9MB 8.0MB/s 
     |████████████████████████████████| 14.7MB 279kB/s 


In [3]:
import folium
from folium import Choropleth, Circle, Marker
from folium.plugins import HeatMap, MarkerCluster

Several arguments customize the appearance of the map:

location sets the initial center of the map. We use the latitude (42.32° N) and longitude (-71.0589° E) of the city of Boston.
tiles changes the styling of the map; in this case, we choose the OpenStreetMap style. If you're curious, you can find the other options listed here.
zoom_start sets the initial level of zoom of the map, where higher values zoom in closer to the map.
Take the time now to explore by zooming in and out, or by dragging the map in different directions.

In [7]:
# Create a map

# location = lat long dc
# tiles changes the styling of the map
m_1 = folium.Map(location=[38.9072,-77.0369], tiles='openstreetmap', zoom_start=10)


# Display the map
m_1

# https://www.kaggle.com/alexisbcook/interactive-maps